In [22]:
!git clone https://github.com/sirimiri13/auto-derain.git
%cd auto-derain
!bash setup.sh /kaggle/working/auto-derain

Cloning into 'auto-derain'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 54 (delta 28), reused 41 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (54/54), 19.85 KiB | 3.97 MiB/s, done.
Resolving deltas: 100% (28/28), done.
/kaggle/working/auto-derain
🚀 Setting up Auto-Derain in /kaggle/working/auto-derain
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.3 which is incompatible.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
cesium 0.12.4 requires numpy<3

In [23]:
RLP_WEIGHTS = "/kaggle/input/unet-checkpoints/Uformer_T_RLP_RPIM.pth"
NERD_WEIGHTS = "/kaggle/input/model_small_rain200l/pytorch/default/1/model_small_Rain200L.pth"
MIX_IMG_DIR="/kaggle/input/data-mix-rain-with-gt/data_mix/test"
MIX_GT_DIR="/kaggle/input/data-mix-rain-with-gt/data_mix/gt"
OUTPUT_DIR_MULTI="/kaggle/working/multi/results/mix"
DATASET_NAME="mix"


In [26]:
!python test.py {MIX_IMG_DIR} {OUTPUT_DIR_MULTI} {RLP_WEIGHTS} {NERD_WEIGHTS} {MIX_GT_DIR} {DATASET_NAME}

Using device: cuda
📂 Using base directory: /kaggle/working/auto-derain
2025-10-30 16:54:21.791994: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761843261.810870    1280 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761843261.816037    1280 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

🌧️ Deraining...
📂 Found 7 images in /kaggle/input/data-mix-rain-with-gt/data_mix/test
🔍 Classifying into day/night...
Classifying: 100%|███████████████████████████████| 7/7 [00:00<00:00,  7.96img/s]
✅ Classification done: 3 day, 4 night
⚙️ Enabling tile mode for night model (max image 1920x1080, free 14.0GB)

🚀 Running NIGHT model: RLP
✅ Night model done in 26.5s

### USE PYTHON

In [11]:
# from classifier import ImgClassifier  
# from evaluate import evaluate

In [12]:
# %env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

# print("🚀 Initializing ImgClassifier...")
# print("(First run downloads CLIP model ~600MB)\n")

# clf = ImgClassifier(base_dir="/kaggle/working/auto-derain")

In [13]:
# OUTPUT_DIR = "/kaggle/working/results/"
# # Process single image
# clf.derain_single(
#     image_path="/kaggle/input/rain13kdataset/test/test/Rain100L/input/11.png",
#     output_dir=OUTPUT_DIR,
#     rlp_weights=RLP_WEIGHTS,
#     nerd_weights=NERD_WEIGHTS,
#     gt_dir="/kaggle/input/rain13kdataset/test/test/Rain100L/target"
# )

In [14]:
# MIX_IMG_DIR="/kaggle/input/data-mix-rain-with-gt/data_mix/test"
# MIX_GT_DIR="/kaggle/input/data-mix-rain-with-gt/data_mix/gt"
# OUTPUT_DIR_MULTI="/kaggle/working/multi/results/"
# DATASET_NAME="mix"
    

# clf.derain_auto(
#     image_dir=MIX_IMG_DIR,
#     output_dir=OUTPUT_DIR_MULTI + DATASET_NAME, 
#     rlp_weights=RLP_WEIGHTS,
#     nerd_weights=NERD_WEIGHTS,
#     gt_dir=MIX_GT_DIR
# )


In [15]:
# !python test.py \
#     /kaggle/input/rain-images \
#     /kaggle/working/derain_results \
#     /kaggle/input/weights/rlp_best.pth \
#     /kaggle/input/weights/nerd_best.pth

In [16]:
# evaluate(
#     root_dir=OUTPUT_DIR_MULTI,
#     gt_root_dir=MIX_GT_DIR,
#     dataset=DATASET_NAME
# )